## **Module**

In [1]:
from wmbio import * 

/home/wmbio/anaconda3/envs/multiomics-cpu/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## **Best Sub-Group Selection**

* **FILE PATH**

In [10]:
os.chdir("..")
os.getcwd()

'/home/wmbio/WORK/gitworking/Multi-omics-intergration'

In [20]:
# PATH
CANCER_TYPE = "BRCA"
GROUP_PHTH = os.getcwd() + '/group/'
PNG_PATH = os.getcwd() + '/png/'
GROUP_VALIDATION_PATH = os.getcwd() + '/group_validation/'
DEG_PATH = os.getcwd() + "/best_deg/"
RDATA_PATH = os.getcwd() + "/RAW_DATA/GDC_PREPARE/"
METHOD = 'deseq2'
LOGFC=1
FDR=0.05

In [15]:
# Load Validation score
col=['FILENAME','Log Rank Test','Silhouette','RNA_ANOVA_F1','RNA_RF_F1',
     'miRNA_ANOVA_F1','miRNA_RF_F1','Methylation_ANOVA_F1','Methylation_RF_F1']

group_score = pd.read_csv(GROUP_VALIDATION_PATH + 'BRCA_validation.csv', usecols=col)

# Condition for Filtering
filter_cond = (group_score['Silhouette'] >= 0.2) & (group_score['Log Rank Test'] < 0.05) & \
              ((group_score['RNA_ANOVA_F1'] > 80) | (group_score['RNA_RF_F1'] > 80)) & \
              ((group_score['miRNA_ANOVA_F1'] > 80) | (group_score['miRNA_RF_F1'] > 80)) & \
              ((group_score['Methylation_ANOVA_F1'] > 85) | (group_score['Methylation_RF_F1'] > 85))
group_score = group_score[filter_cond].sort_values(["Silhouette"], ascending = (False))

bestSubgroup = group_score.FILENAME.to_list()

# 5 fold로 제한.
if len(bestSubgroup) > 10:
    bestSubgroup = bestSubgroup[:10]

In [16]:
bestSubgroup

['20220304-184536',
 '20220320-011639',
 '20220320-012539',
 '20220315-162103',
 '20220319-160047',
 '20220320-204635',
 '20220317-163050',
 '20220316-221634',
 '20220311-154105',
 '20220319-215519']

In [25]:
METHOD

'deseq2'

In [33]:
DEG_CHECK = "_".join([CANCER_TYPE, METHOD.upper(), bestSubgroup[0]]) + ".txt"

In [32]:
DEG_PATH + CANCER_TYPE

'/home/wmbio/WORK/gitworking/Multi-omics-intergration/best_deg/BRCA'

In [34]:
os.path.isfile(DEG_PATH + CANCER_TYPE + "/" + DEG_CHECK) 

True

In [37]:
pd.read_csv(DEG_PATH + CANCER_TYPE + "/" + DEG_CHECK, sep = "\t")

,row,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,NAT1,7705.240699,-1.970460,0.123975,-15.893965,6.977178e-57,2.133698e-55
1,ABAT,3844.197065,-1.509023,0.088463,-17.058272,3.033971e-65,1.448235e-63
2,ABP1,223.403991,2.124583,0.173080,12.275169,1.231349e-34,1.168056e-33
3,ACACB,1572.979236,-1.043797,0.083287,-12.532482,4.958604e-36,5.051196e-35
4,ACADSB,6202.049094,-1.045333,0.072554,-14.407596,4.635650e-47,8.597601e-46
...,...,...,...,...,...,...,...
1085,FAM72A,102.879465,1.363876,0.085020,16.041873,6.515904e-58,2.086911e-56
1086,SNORD116-28,75.248228,-1.133515,0.095882,-11.822042,3.002945e-32,2.491524e-31
1087,FAM47E,313.735255,-1.146521,0.072873,-15.733168,8.961300e-56,2.586446e-54
1088,GAGE12D,261.656401,6.809779,0.947321,7.188461,6.552555e-13,1.896002e-12


In [38]:
# DEA
dea_result = list()
for best_group in bestSubgroup:
    
    DEG_CHECK = "_".join([CANCER_TYPE, METHOD.upper(), best_group]) + ".txt"
    SAMPLE_GROUP = GROUP_PHTH + CANCER_TYPE + "/" + CANCER_TYPE + "_GROUP_" + best_group + ".txt"
    
    if os.path.isfile(DEG_PATH + CANCER_TYPE + "/" + DEG_CHECK):
        deg_list = pd.read_csv(DEG_PATH + CANCER_TYPE + "/" + DEG_CHECK, sep = "\t")
    else :
        # DEG Extraction
        deg_list = deg_extract(log_fc=LOGFC, fdr=FDR,
                       cancer_type=CANCER_TYPE, 
                       sample_group=SAMPLE_GROUP, deg_path=DEG_PATH, 
                       file_name=best_group,
                       rdata_path=RDATA_PATH,
                       method=METHOD,
                       batch_removal=True)
    
    dea_result.append(deg_list)

In [40]:
# Filter DEA
# combine result
if METHOD == 'all':
    dea_combine = list(map(deseq2_edger_combine, dea_result))
    dea_combine = [col_rename(dea_combine[index], index, bestSubgroup) for index in range(len(dea_combine))]
    dea_combine = reduce(lambda left, right : pd.merge(left, right, left_on='gene', right_on='gene', how = 'outer'), dea_combine)
elif METHOD == 'deseq2' :
    dea_combine = list(map(lambda d : d[["row", "log2FoldChange"]], dea_result))
    dea_combine = [col_rename(dea_combine[index], index, bestSubgroup) for index in range(len(dea_combine))]
    dea_combine = reduce(lambda left, right : pd.merge(left, right, left_on='gene', right_on='gene', how = 'inner'), dea_combine)

# median & mean
dea_combine["SubGroup-log2FC_median"] = dea_combine.iloc[:, 1:].median(axis=1)
dea_combine["SubGroup-log2FC_mean"] = dea_combine.iloc[:, 1:].mean(axis=1)

* **NT vs TP DEA(Deseq2)**

In [11]:
nt_tp_deseq2 = deg_extract_normal(log_fc=0, pvalue=0.25, cancer_type=CANCER_TYPE, 
                                  rdata_path=RDATA_PATH, deg_path=DEG_PATH, batch_removal=True)

nt_tp_deseq2_col = nt_tp_deseq2[['row', 'log2FoldChange', 'pvalue']]
nt_tp_deseq2_col.columns = ['gene', 'NT-TP_log2FoldChange', 'pvalue']

Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  

* **Write Result**

In [12]:
pd.merge(left=dea_combine, right=nt_tp_deseq2_col, left_on='gene', right_on='gene', how = 'left').to_csv('temp.csv', index = False)